In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import re
import sys
import os
import mne
import mne_connectivity
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da 
import itertools
from plotnine import *
import pickle
import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest
import tensorpac as tp
from scipy.ndimage import label

In [3]:
## Prep paths ##

raw_data_dir = f"/home/brooke/pacman/raw_data/"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/"
t7_folder = '/media/brooke/T7/pacman/connectivity/pac/data/to_ofc'

## Functions

In [4]:
def find_largest_cluster_mask(matrix):

    if not matrix.any():
        return matrix
    
    # Label the connected components in the inverted matrix
    labeled_array, num_features = label(matrix)
    
    # Find the size of each connected component
    component_sizes = np.bincount(labeled_array.ravel())
    
    # Exclude the background component (label 0)
    largest_component_label = component_sizes[1:].argmax() + 1
    
    # Create a mask for the largest component
    largest_cluster_mask = (labeled_array == largest_component_label).astype(int)
    
    return largest_cluster_mask

In [5]:
def find_pair_pac_pvalue(p):

    # create average pac and average perm pac by averaging over trials
    average_perm_pac = p.surrogates.mean(-1)
    average_pac = p.pac.mean(-1)

    # find the uncorrected p values
    tmp = average_pac <= average_perm_pac
    uncorrected_p = np.sum(tmp, axis = 0)/200

    # create a mask of largest sig cluster
    mask = uncorrected_p.copy()
    mask[uncorrected_p > .01] = 0
    mask[uncorrected_p <= .01] = 1
    large_cluster = find_largest_cluster_mask(mask)

    if np.sum(large_cluster) >= 5:

        # sum pac values in the largest cluster for each surrogate
        perm_test_stat = np.zeros((200, 1))
        for i in range(200):
            tmp = average_perm_pac[i, :, :]
            tmp[large_cluster == 0] = 0
            perm_test_stat[i] = np.sum(tmp)  

        # get summed pac value of the real data
        real_pac = average_pac.copy()
        real_pac[large_cluster == 0] = 0
        test_stat = np.sum(real_pac) 

        # get the p value
        pval = np.sum(test_stat <= perm_test_stat)/200
    else:
        # get summed pac value of the real data
        real_pac = average_pac.copy()
        real_pac[large_cluster == 0] = 0
        test_stat = np.sum(real_pac) 
        pval = 1        


    return test_stat, pval, large_cluster

In [6]:
sg_full_df = pd.read_csv('/home/brooke/pacman/connectivity/scripts/pac/sig_theta_pairs.csv')

# Filter the DataFrame
sg_df = sg_full_df[
    (sg_full_df['roi_pair'].isin(['ofc_insula', 'ofc_amyg', 'ofc_hc', 'ofc_dlpfc', 'ofc_cing'])) &
    (sg_full_df['metric'] == 'Imaginary Coherence')
]

# Add elec1 and elec2 columns
sg_df['elec1'] = sg_df['pairs'].str.replace('_to_.*', '', regex=True)
sg_df['elec2'] = sg_df['pairs'].str.replace('.*_to_', '', regex=True)


/tmp/ipykernel_45580/991895224.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_45580/991895224.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
sg_df

,subject,pairs,roi_pair,metric,elec1,elec2
77,BJH025,A1-A2_to_B10-B11,ofc_cing,Imaginary Coherence,A1-A2,B10-B11
80,BJH025,A1-A2_to_B12-B13,ofc_dlpfc,Imaginary Coherence,A1-A2,B12-B13
83,BJH025,A1-A2_to_B13-B14,ofc_dlpfc,Imaginary Coherence,A1-A2,B13-B14
86,BJH025,A1-A2_to_B14-B15,ofc_dlpfc,Imaginary Coherence,A1-A2,B14-B15
89,BJH025,A1-A2_to_B15-B16,ofc_dlpfc,Imaginary Coherence,A1-A2,B15-B16
...,...,...,...,...,...,...
14340,LL13,LOF1-LOF2_to_RH1-RH2,ofc_hc,Imaginary Coherence,LOF1-LOF2,RH1-RH2
14343,LL10,ROF1-ROF2_to_RH1-RH2,ofc_hc,Imaginary Coherence,ROF1-ROF2,RH1-RH2
14346,LL12,ROF1-ROF2_to_RH1-RH2,ofc_hc,Imaginary Coherence,ROF1-ROF2,RH1-RH2
14348,LL13,ROF1-ROF2_to_RH1-RH2,ofc_hc,Imaginary Coherence,ROF1-ROF2,RH1-RH2


### From OFC Results - Approach

In [8]:
# intialize results pd
results = pd.DataFrame(columns = ['subject', 'elec1', 'elec2', 'test_stat', 'pval'])

# get all the files
t7_folder = '/media/brooke/T7/pacman/connectivity/pac/approach/from_ofc'
all_files = os.listdir(t7_folder)
pac_files= list(filter(lambda s: '_to_' in s, all_files))

# loop through all the files
for file in pac_files:
    with open(f'{t7_folder}/{file}', 'rb') as f:
        p = pickle.load(f)

    # calculate p value
    test_stat, pval, cluster= find_pair_pac_pvalue(p)

    # get subject and electrode info
    sub, elec1, _, elec2, _ = file.split("_")

    # add to results
    results = pd.concat([results, pd.DataFrame({'subject': [sub], 'elec1': [elec1], 'elec2': [elec2], 'test_stat': [test_stat], 'pval': [pval]})])



/tmp/ipykernel_45580/3801739472.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [13]:
results

,subject,elec1,elec2,test_stat,pval
0,BJH025,A1-A2,B10-B11,0.048114,0.005
0,BJH025,A1-A2,B12-B13,0.120658,0.000
0,BJH025,A1-A2,B13-B14,0.013097,1.000
0,BJH025,A1-A2,B14-B15,0.044039,0.000
0,BJH025,A1-A2,B15-B16,0.122112,0.000
...,...,...,...,...,...
0,SLCH002,B8-B9,i2-i3,0.000000,1.000
0,SLCH002,B9-B10,i2-i3,0.113613,0.000
0,SLCH002,B9-B10,i3-i6,0.000000,1.000
0,SLCH002,B6-B7,i3-i6,0.000000,1.000


In [10]:
# left join with sg_df
compiled_results = results.merge(sg_df, left_on = ['subject', 'elec1', 'elec2'], right_on = ['subject', 'elec1', 'elec2'], how = 'left')

In [11]:
compiled_results

,subject,elec1,elec2,test_stat,pval,pairs,roi_pair,metric
0,BJH025,A1-A2,B10-B11,0.048114,0.005,A1-A2_to_B10-B11,ofc_cing,Imaginary Coherence
1,BJH025,A1-A2,B12-B13,0.120658,0.000,A1-A2_to_B12-B13,ofc_dlpfc,Imaginary Coherence
2,BJH025,A1-A2,B13-B14,0.013097,1.000,A1-A2_to_B13-B14,ofc_dlpfc,Imaginary Coherence
3,BJH025,A1-A2,B14-B15,0.044039,0.000,A1-A2_to_B14-B15,ofc_dlpfc,Imaginary Coherence
4,BJH025,A1-A2,B15-B16,0.122112,0.000,A1-A2_to_B15-B16,ofc_dlpfc,Imaginary Coherence
...,...,...,...,...,...,...,...,...
1651,SLCH002,B8-B9,i2-i3,0.000000,1.000,B8-B9_to_i2-i3,ofc_amyg,Imaginary Coherence
1652,SLCH002,B9-B10,i2-i3,0.113613,0.000,B9-B10_to_i2-i3,ofc_amyg,Imaginary Coherence
1653,SLCH002,B9-B10,i3-i6,0.000000,1.000,B9-B10_to_i3-i6,ofc_amyg,Imaginary Coherence
1654,SLCH002,B6-B7,i3-i6,0.000000,1.000,B6-B7_to_i3-i6,ofc_amyg,Imaginary Coherence


In [12]:
# save results
compiled_results.to_csv('/home/brooke/pacman/connectivity/scripts/pac/compiled_approach_from_ofc_results.csv', index = False)

In [14]:
# Perform the anti-join
merged_df = sg_df.merge(compiled_results, on=['subject', 'elec1', 'elec2'], how='left', indicator=True)

# Filter the rows that are only in sg_df
remaining_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])

# fix and drop columns
remaining_df.drop(['test_stat', 'pval', 'pairs_y', 'roi_pair_y', 'metric_y'], axis=1, inplace=True)
rename_mapping = dict(zip(remaining_df.columns, sg_df.columns))
remaining_df.rename(columns=rename_mapping, inplace=True)
remaining_df

,subject,pairs,roi_pair,metric,elec1,elec2
1153,BJH027,AL1-AL2_to_BL1-BL2,ofc_amyg,Imaginary Coherence,AL1-AL2,BL1-BL2
1154,BJH027,AL10-AL11_to_BL1-BL2,ofc_amyg,Imaginary Coherence,AL10-AL11,BL1-BL2
1155,BJH027,AL11-AL12_to_BL1-BL2,ofc_amyg,Imaginary Coherence,AL11-AL12,BL1-BL2
1156,BJH027,AL13-AL14_to_BL1-BL2,ofc_amyg,Imaginary Coherence,AL13-AL14,BL1-BL2
1157,BJH027,AL2-AL3_to_BL1-BL2,ofc_amyg,Imaginary Coherence,AL2-AL3,BL1-BL2
...,...,...,...,...,...,...
1328,SLCH002,B8-B9_to_i2-i3,ofc_amyg,Imaginary Coherence,B8-B9,i2-i3
1329,SLCH002,B9-B10_to_i2-i3,ofc_amyg,Imaginary Coherence,B9-B10,i2-i3
1330,SLCH002,B6-B7_to_i3-i6,ofc_amyg,Imaginary Coherence,B6-B7,i3-i6
1331,SLCH002,B9-B10_to_i3-i6,ofc_amyg,Imaginary Coherence,B9-B10,i3-i6


### To OFC  - Approach


In [14]:
# intialize results pd
results = pd.DataFrame(columns = ['subject', 'elec1', 'elec2', 'test_stat', 'pval'])

# get all the files
t7_folder = '/media/brooke/T7/pacman/connectivity/pac/approach/to_ofc'
all_files = os.listdir(t7_folder)
pac_files= list(filter(lambda s: '_to_' in s, all_files))

# loop through all the files
for file in pac_files:
    with open(f'{t7_folder}/{file}', 'rb') as f:
        p = pickle.load(f)

    # calculate p value
    test_stat, pval, cluster= find_pair_pac_pvalue(p)

    # get subject and electrode info
    sub, elec1, _, elec2, _ = file.split("_")

    # add to results
    # print(f"file: {file}, test stat: {test_stat}, pval: {pval}")
    results = pd.concat([results, pd.DataFrame({'subject': [sub], 'elec1': [elec2], 'elec2': [elec1], 'test_stat': [test_stat], 'pval': [pval]})])



/tmp/ipykernel_45580/2687791482.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [15]:
# left join with sg_df
compiled_results = results.merge(sg_df, left_on = ['subject', 'elec1', 'elec2'], right_on = ['subject', 'elec1', 'elec2'], how = 'left')

In [16]:
compiled_results

,subject,elec1,elec2,test_stat,pval,pairs,roi_pair,metric
0,BJH025,A1-A2,B10-B11,0.016242,1,A1-A2_to_B10-B11,ofc_cing,Imaginary Coherence
1,BJH025,A1-A2,B12-B13,0.015757,1,A1-A2_to_B12-B13,ofc_dlpfc,Imaginary Coherence
2,BJH025,A1-A2,B13-B14,0.020302,1,A1-A2_to_B13-B14,ofc_dlpfc,Imaginary Coherence
3,BJH025,A1-A2,B14-B15,0.008129,1,A1-A2_to_B14-B15,ofc_dlpfc,Imaginary Coherence
4,BJH025,A1-A2,B15-B16,0.089842,0.0,A1-A2_to_B15-B16,ofc_dlpfc,Imaginary Coherence
...,...,...,...,...,...,...,...,...
1651,SLCH002,B8-B9,i2-i3,0.005808,1,B8-B9_to_i2-i3,ofc_amyg,Imaginary Coherence
1652,SLCH002,B9-B10,i2-i3,0.019015,1,B9-B10_to_i2-i3,ofc_amyg,Imaginary Coherence
1653,SLCH002,B9-B10,i3-i6,0.000000,1,B9-B10_to_i3-i6,ofc_amyg,Imaginary Coherence
1654,SLCH002,B6-B7,i3-i6,0.012027,1,B6-B7_to_i3-i6,ofc_amyg,Imaginary Coherence


In [17]:
# save results
compiled_results.to_csv('/home/brooke/pacman/connectivity/scripts/pac/compiled_approach_to_ofc_results.csv', index = False)

### From OFC Results - Avoid

In [ ]:
# intialize results pd
results = pd.DataFrame(columns = ['subject', 'elec1', 'elec2', 'test_stat', 'pval'])

# get all the files
t7_folder = '/media/brooke/T7/pacman/connectivity/pac/avoid/from_ofc'
all_files = os.listdir(t7_folder)
pac_files= list(filter(lambda s: '_to_' in s, all_files))

# loop through all the files
for file in pac_files:
    with open(f'{t7_folder}/{file}', 'rb') as f:
        p = pickle.load(f)

    # calculate p value
    test_stat, pval, cluster= find_pair_pac_pvalue(p)

    # get subject and electrode info
    sub, elec1, _, elec2, _ = file.split("_")

    # add to results
    results = pd.concat([results, pd.DataFrame({'subject': [sub], 'elec1': [elec1], 'elec2': [elec2], 'test_stat': [test_stat], 'pval': [pval]})])



/tmp/ipykernel_530533/3055721220.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [ ]:
# left join with sg_df
compiled_results = results.merge(sg_df, left_on = ['subject', 'elec1', 'elec2'], right_on = ['subject', 'elec1', 'elec2'], how = 'left')

In [ ]:
compiled_results

,subject,elec1,elec2,test_stat,pval,pairs,roi_pair,metric
0,BJH025,A1-A2,B10-B11,0.048114,0.005,A1-A2_to_B10-B11,ofc_cing,Imaginary Coherence
1,BJH025,A1-A2,B12-B13,0.120658,0.000,A1-A2_to_B12-B13,ofc_dlpfc,Imaginary Coherence
2,BJH025,A1-A2,B13-B14,0.013097,1.000,A1-A2_to_B13-B14,ofc_dlpfc,Imaginary Coherence
3,BJH025,A1-A2,B14-B15,0.044039,0.000,A1-A2_to_B14-B15,ofc_dlpfc,Imaginary Coherence
4,BJH025,A1-A2,B15-B16,0.122112,0.000,A1-A2_to_B15-B16,ofc_dlpfc,Imaginary Coherence
...,...,...,...,...,...,...,...,...
1323,LL19,ROF2-ROF3,RAC5-RAC6,0.059247,0.000,ROF2-ROF3_to_RAC5-RAC6,ofc_dlpfc,Imaginary Coherence
1324,LL19,ROF2-ROF3,ROF10-ROF11,0.073543,0.005,ROF2-ROF3_to_ROF10-ROF11,ofc_dlpfc,Imaginary Coherence
1325,LL19,ROF2-ROF3,ROF11-ROF12,0.014722,1.000,ROF2-ROF3_to_ROF11-ROF12,ofc_dlpfc,Imaginary Coherence
1326,LL19,ROF3-ROF4,RAC4-RAC5,0.017790,1.000,ROF3-ROF4_to_RAC4-RAC5,ofc_dlpfc,Imaginary Coherence


In [ ]:
# save results
compiled_results.to_csv('/home/brooke/pacman/connectivity/scripts/pac/compiled_avoidance_from_ofc_results.csv', index = False)

### To OFC  - Avoid


In [ ]:
# intialize results pd
results = pd.DataFrame(columns = ['subject', 'elec1', 'elec2', 'test_stat', 'pval'])

# get all the files
t7_folder = '/media/brooke/T7/pacman/connectivity/pac/avoid/to_ofc'
all_files = os.listdir(t7_folder)
pac_files= list(filter(lambda s: '_to_' in s, all_files))

# loop through all the files
for file in pac_files:
    with open(f'{t7_folder}/{file}', 'rb') as f:
        p = pickle.load(f)

    # calculate p value
    test_stat, pval, cluster= find_pair_pac_pvalue(p)

    # get subject and electrode info
    sub, elec1, _, elec2, _ = file.split("_")

    # add to results
    # print(f"file: {file}, test stat: {test_stat}, pval: {pval}")
    results = pd.concat([results, pd.DataFrame({'subject': [sub], 'elec1': [elec2], 'elec2': [elec1], 'test_stat': [test_stat], 'pval': [pval]})])



/tmp/ipykernel_530533/2970386743.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


In [ ]:
# left join with sg_df
compiled_results = results.merge(sg_df, left_on = ['subject', 'elec1', 'elec2'], right_on = ['subject', 'elec1', 'elec2'], how = 'left')

In [ ]:
compiled_results

,subject,elec1,elec2,test_stat,pval,pairs,roi_pair,metric
0,BJH025,A1-A2,B10-B11,0.016242,1,A1-A2_to_B10-B11,ofc_cing,Imaginary Coherence
1,BJH025,A1-A2,B12-B13,0.015757,1,A1-A2_to_B12-B13,ofc_dlpfc,Imaginary Coherence
2,BJH025,A1-A2,B13-B14,0.020302,1,A1-A2_to_B13-B14,ofc_dlpfc,Imaginary Coherence
3,BJH025,A1-A2,B14-B15,0.008129,1,A1-A2_to_B14-B15,ofc_dlpfc,Imaginary Coherence
4,BJH025,A1-A2,B15-B16,0.089842,0.0,A1-A2_to_B15-B16,ofc_dlpfc,Imaginary Coherence
...,...,...,...,...,...,...,...,...
1651,SLCH002,B8-B9,i2-i3,0.005808,1,B8-B9_to_i2-i3,ofc_amyg,Imaginary Coherence
1652,SLCH002,B9-B10,i2-i3,0.019015,1,B9-B10_to_i2-i3,ofc_amyg,Imaginary Coherence
1653,SLCH002,B9-B10,i3-i6,0.000000,1,B9-B10_to_i3-i6,ofc_amyg,Imaginary Coherence
1654,SLCH002,B6-B7,i3-i6,0.012027,1,B6-B7_to_i3-i6,ofc_amyg,Imaginary Coherence


In [ ]:
# save results
compiled_results.to_csv('/home/brooke/pacman/connectivity/scripts/pac/compiled_avoidance_to_ofc_results.csv', index = False)